In [1]:
import os
import tensorflow as tf
import numpy as np
from megnet.utils.models import load_model

print("=" * 60)
print("🔍 GPU DEBUG VE TEST SCRİPTİ")
print("=" * 60)

# 1. TensorFlow GPU kontrolü
print("\n1️⃣ TensorFlow GPU Durumu:")
print(f"TensorFlow versionu: {tf.__version__}")
print(f"CUDA build: {tf.test.is_built_with_cuda()}")
print(f"GPU kullanılabilir: {tf.test.is_gpu_available()}")

# 2. Fiziksel cihazlar
print("\n2️⃣ Fiziksel Cihazlar:")
physical_devices = tf.config.list_physical_devices()
for device in physical_devices:
    print(f"  - {device}")

# 3. GPU cihazlar detayı
print("\n3️⃣ GPU Cihazları:")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for i, gpu in enumerate(gpus):
        print(f"  GPU {i}: {gpu}")
        try:
            # GPU memory growth ayarı
            tf.config.experimental.set_memory_growth(gpu, True)
            print(f"    ✅ Memory growth etkinleştirildi")
        except Exception as e:
            print(f"    ❌ Memory growth hatası: {e}")
else:
    print("  ❌ GPU bulunamadı!")

# 4. Logical devices
print("\n4️⃣ Logical Cihazlar:")
logical_devices = tf.config.list_logical_devices()
for device in logical_devices:
    print(f"  - {device}")

# 5. CUDA kütüphaneleri kontrolü
print("\n5️⃣ CUDA Kütüphaneleri:")
try:
    from tensorflow.python.platform import build_info
    print(f"CUDA version: {build_info.build_info['cuda_version']}")
    print(f"cuDNN version: {build_info.build_info['cudnn_version']}")
except:
    print("CUDA bilgileri alınamadı")

# 6. GPU test işlemi
print("\n6️⃣ GPU Test İşlemi:")
try:
    with tf.device('/GPU:0'):
        # Basit bir GPU işlemi
        a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
        b = tf.constant([[1.0, 1.0], [0.0, 1.0]])
        c = tf.matmul(a, b)
        print(f"  ✅ GPU işlem başarılı: {c.numpy()}")
        print(f"  📍 İşlem cihazı: {c.device}")
except Exception as e:
    print(f"  ❌ GPU işlem hatası: {e}")

# 7. MEGNet model yükleme testi
print("\n7️⃣ MEGNet Model Testi:")
try:
    print("  🔄 Model yükleniyor...")
    model = load_model("Bandgap_MP_2018")
    print("  ✅ Model yüklendi")
    
    # Model device kontrolü
    print("  📍 Model cihaz bilgisi:")
    for layer in model.model.layers[:3]:  # İlk 3 layer
        print(f"    {layer.name}: {layer.dtype}")
        
except Exception as e:
    print(f"  ❌ Model yükleme hatası: {e}")

# 8. Ortam değişkenleri
print("\n8️⃣ Önemli Ortam Değişkenleri:")
env_vars = [
    'CUDA_VISIBLE_DEVICES',
    'TF_CPP_MIN_LOG_LEVEL', 
    'TF_FORCE_GPU_ALLOW_GROWTH'
]
for var in env_vars:
    value = os.environ.get(var, 'Ayarlanmamış')
    print(f"  {var}: {value}")

# 9. Memory test
print("\n9️⃣ GPU Memory Testi:")
if gpus:
    try:
        # Büyük tensor oluştur
        with tf.device('/GPU:0'):
            big_tensor = tf.random.normal([1000, 1000])
            result = tf.reduce_sum(big_tensor)
            print(f"  ✅ Büyük tensor işlemi başarılı: {result.numpy()}")
    except Exception as e:
        print(f"  ❌ Memory test hatası: {e}")

print("\n" + "=" * 60)
print("🏁 DEBUG TAMAMLANDI")
print("=" * 60)

🔍 GPU DEBUG VE TEST SCRİPTİ

1️⃣ TensorFlow GPU Durumu:
TensorFlow versionu: 2.8.4
CUDA build: True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU kullanılabilir: False

2️⃣ Fiziksel Cihazlar:
  - PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')

3️⃣ GPU Cihazları:
  ❌ GPU bulunamadı!

4️⃣ Logical Cihazlar:
  - LogicalDevice(name='/device:CPU:0', device_type='CPU')

5️⃣ CUDA Kütüphaneleri:
CUDA version: 64_112
cuDNN version: 64_8

6️⃣ GPU Test İşlemi:
  ✅ GPU işlem başarılı: [[1. 3.]
 [3. 7.]]
  📍 İşlem cihazı: /job:localhost/replica:0/task:0/device:CPU:0

7️⃣ MEGNet Model Testi:
  🔄 Model yükleniyor...
  ✅ Model yüklendi
  📍 Model cihaz bilgisi:
    input_1: int32
    embedding_1: float32
    input_2: float32

8️⃣ Önemli Ortam Değişkenleri:
  CUDA_VISIBLE_DEVICES: Ayarlanmamış
  TF_CPP_MIN_LOG_LEVEL: Ayarlanmamış
  TF_FORCE_GPU_ALLOW_GROWTH: Ayarlanmamış

9️⃣ GPU Memory Testi:

🏁 DEBUG TAMAMLANDI


In [2]:
def gaussian_expansion(distances, centers, width):
    # distances: (n_bond,)
    # centers: (num_centers,)
    diff = distances[:, None] - centers[None, :]  # (n_bond, num_centers)
    return np.exp(-0.5 * (diff / width)**2)

Denemek için tek bir embedding alalım

In [4]:
from pymatgen.core import Structure
structure = Structure.from_file(str("data/cif_indirme/mp-2.cif"))

In [5]:
from megnet.utils.models import load_model
from pymatgen.core import Structure
import numpy as np

# Modeli yükle
model = load_model("Bandgap_MP_2018")

# Manuel graph conversion
from megnet.data.graph import GaussianDistance
from megnet.data.crystal import CrystalGraph



cutoff = 5.0  # model genellikle 4.0 veya 5.0 cutoff kullanır, 5.0 daha garanti


gaussian_centers = np.linspace(0, cutoff + 1, 100)
gaussian_width = 0.5
gd = GaussianDistance(gaussian_centers, gaussian_width)


# Convert
graph_manual = model.graph_converter.convert(structure)
graph_input_manual = [np.expand_dims(x, axis=0) for x in graph_manual]


bond_distances = graph_manual['bond']  # (n_bond,)
bond_features = gaussian_expansion(bond_distances, gd.centers, gd.width)  # (n_bond, 100)

n_atom = len(graph_manual['atom'])
n_bond = len(graph_manual['bond'])

atom_graph = np.zeros(n_atom, dtype=int)
bond_graph = np.zeros(n_bond, dtype=int)

graph_input_manual = [
    np.expand_dims(np.array(graph_manual['atom']), axis=0),       # (1, n_atom)
    np.expand_dims(bond_features.astype(np.float32), axis=0),    # (1, n_bond, 100)
    np.expand_dims(graph_manual['state'].astype(np.float32), axis=0),  # (1, d_state)
    np.expand_dims(np.array(graph_manual['index1']), axis=0),    # (1, n_bond)
    np.expand_dims(np.array(graph_manual['index2']), axis=0),    # (1, n_bond)
    np.expand_dims(atom_graph, axis=0),                          # (1, n_atom)
    np.expand_dims(bond_graph, axis=0),                          # (1, n_bond)
]



"""
# Doğru graph_converter (model ile uyumlu!)
graph = model.graph_converter.convert(structure)

# Girdileri hazırla (7 input!)
graph_input = [np.expand_dims(x, axis=0) for x in graph]
"""
# Ara katmandan embedding modeli oluştur
from tensorflow.keras.models import Model
embedding_model = Model(
    inputs=model.model.inputs,
    outputs=model.model.get_layer("concatenate_1").output
)

# Predict
embedding = embedding_model.predict(graph_input_manual)
print("Embedding shape:", embedding.shape)

Embedding shape: (1, 1, 96)


In [14]:
import os
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import List, Optional, Tuple
from pathlib import Path
import logging
from pymatgen.core import Structure
from megnet.utils.models import load_model
from megnet.data.graph import GaussianDistance
from tensorflow.keras.models import Model

# === Logging setup ===
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# === GPU / CPU Setup ===
def setup_device():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logger.info(f"✅ GPU found: {len(gpus)}. Using GPU.")
            return "/GPU:0"
        except RuntimeError as e:
            logger.error(f"❌ GPU setup error: {e}")
    
    logger.warning("⚠️ No GPU found or error occurred. Switching to CPU.")
    return "/CPU:0"

# === Gaussian Expansion ===
def gaussian_expansion(distances: np.ndarray, centers: np.ndarray, width: float) -> np.ndarray:
    return np.exp(-((distances[:, None] - centers[None, :]) ** 2) / width ** 2)

# === Graph Preparation ===
def prepare_graph_input(structure: Structure, model, cutoff: float = 5.0, width: float = 0.5, num_centers: int = 100) -> List[np.ndarray]:
    gaussian_centers = np.linspace(0, cutoff + 1, num_centers)
    gd = GaussianDistance(gaussian_centers, width)
    graph_manual = model.graph_converter.convert(structure)
    bond_distances = graph_manual['bond']
    bond_features = gaussian_expansion(bond_distances, gd.centers, gd.width)
    
    return [
        np.expand_dims(np.array(graph_manual['atom']), axis=0),
        np.expand_dims(bond_features.astype(np.float32), axis=0),
        np.expand_dims(graph_manual['state'].astype(np.float32), axis=0),
        np.expand_dims(np.array(graph_manual['index1']), axis=0),
        np.expand_dims(np.array(graph_manual['index2']), axis=0),
        np.expand_dims(np.zeros(len(graph_manual['atom']), dtype=int), axis=0),
        np.expand_dims(np.zeros(len(graph_manual['bond']), dtype=int), axis=0),
    ]

# === Main Extraction Function ===
def extract_embeddings_from_cif_folder(cif_folder_path: str, model, embedding_model, device: str,
                                      batch_size: int = 16, autosave_interval: int = 1000,
                                      output_dir: str = "autosaves") -> Tuple[np.ndarray, List[str], List[int]]:
    
    cif_folder = Path(cif_folder_path)
    if not cif_folder.exists():
        raise FileNotFoundError(f"Folder not found: {cif_folder_path}")
    
    cif_files = list(cif_folder.glob("*.cif"))
    logger.info(f"📁 Found {len(cif_files)} CIF files.")
    
    embeddings = []
    material_ids = []
    failed_indices = []
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    for i, cif_file in enumerate(tqdm(cif_files, desc="🚀 Processing CIFs")):
        mid = cif_file.stem
        try:
            struct = Structure.from_file(str(cif_file))
            graph_in = prepare_graph_input(struct, model)
            
            with tf.device(device):
                emb = embedding_model(graph_in, training=False).numpy()
                embeddings.append(emb[0])
                material_ids.append(mid)
                
        except Exception as e:
            logger.warning(f"❌ Failed: {mid} | Error: {e}")
            failed_indices.append(i)
        
        # Periodic Memory Cleanup & Autosave
        if (i + 1) % autosave_interval == 0:
            part_path = Path(output_dir) / f"part_{i//autosave_interval}.npy"
            np.save(part_path, np.array(embeddings))
            tf.keras.backend.clear_session()
            logger.info(f"💾 Autosaved to {part_path}")

    return np.array(embeddings), material_ids, failed_indices

# === Main Runner ===
def main():
    # SETTINGS - Use raw string for Windows paths
    CIF_DIR = r"data\cif_indirme" 
    OUT_FILE = "megnet_embeddings.npy"
    OUT_CSV = "megnet_embeddings.csv"
    
    device = setup_device()
    
    try:
        # Load Model
        megnet_base = load_model("Bandgap_MP_2018")
        embedding_model = Model(inputs=megnet_base.model.inputs, 
                                outputs=megnet_base.model.get_layer("concatenate_1").output)
        
        # Run Extraction
        embs, ids, fails = extract_embeddings_from_cif_folder(CIF_DIR, megnet_base, embedding_model, device)
        
        # Final Save
        np.save(OUT_FILE, embs)
        df = pd.DataFrame(embs)
        df.insert(0, "material_id", ids)
        df.to_csv(OUT_CSV, index=False)
        
        logger.info(f"✅ Done! Saved {len(ids)} embeddings. Fails: {len(fails)}")

    except Exception as e:
        logger.error(f"💥 Critical error: {e}")

if __name__ == "__main__":
    main()

2026-02-03 12:23:57,582 - WARNING - ⚠️ No GPU found or error occurred. Switching to CPU.
2026-02-03 12:23:57,582 - INFO - Package-level mvl_models not included, trying temperary mvl_models downloads..
2026-02-03 12:23:57,582 - INFO - Model found in local mvl_models path
2026-02-03 12:23:58,661 - INFO - 📁 Found 123634 CIF files.
🚀 Processing CIFs:   1%|          | 626/123634 [00:24<1:24:17, 24.32it/s]2026-02-03 12:24:23,615 - WARNING - Isolated atoms found in the structure. The cutoff radius might be small
2026-02-03 12:24:23,692 - WARNING - Isolated atoms found in the structure. The cutoff radius might be small
🚀 Processing CIFs:   1%|          | 629/123634 [00:25<1:22:30, 24.84it/s]2026-02-03 12:24:23,731 - WARNING - Isolated atoms found in the structure. The cutoff radius might be small
2026-02-03 12:24:23,769 - WARNING - Isolated atoms found in the structure. The cutoff radius might be small
2026-02-03 12:24:23,808 - WARNING - Isolated atoms found in the structure. The cutoff radius

In [15]:
df = np.load("autosaves/part_0.npy",allow_pickle=True)
print(df.shape)
df

(998, 1, 96)


array([[[ 0.0000000e+00,  0.0000000e+00,  1.0181684e-03, ...,
         -8.7760315e+00, -1.8966659e+00, -1.0931902e+01]],

       [[ 3.9001369e-01,  0.0000000e+00,  0.0000000e+00, ...,
         -2.4569706e+01, -3.6423311e+00, -1.2085121e+01]],

       [[ 4.6083193e-02, -3.9049979e-02, -1.4455369e-01, ...,
         -6.8950434e+00, -4.9459085e+00, -3.7562966e+00]],

       ...,

       [[ 1.8321717e-01, -1.3546546e-01,  0.0000000e+00, ...,
         -4.3465953e+00, -2.7632823e+00, -6.1148810e+00]],

       [[ 8.7295130e-02,  0.0000000e+00,  0.0000000e+00, ...,
         -6.6389747e+00, -6.3188434e+00, -6.7386723e-01]],

       [[ 6.1140940e-02,  0.0000000e+00,  0.0000000e+00, ...,
         -8.1133490e+00, -6.0890656e+00,  7.8554153e-03]]], dtype=float32)

In [ ]:
import pandas as pd
import numpy as np
import joblib
import os

# Dosyaların bulunduğu klasör yolu
dosya_klasoru = r"autosaves" 

# Yalnızca Part 23 dosyalarının adları
metadata_dosya_adi = "megnet_embeddings_part_23.metadata.pkl"
embeddings_dosya_adi = "megnet_embeddings_part_23.npy"

metadata_yolu = os.path.join(dosya_klasoru, metadata_dosya_adi)
embeddings_yolu = os.path.join(dosya_klasoru, embeddings_dosya_adi)

if os.path.exists(metadata_yolu) and os.path.exists(embeddings_yolu):
    try:
        # Metadata dosyasını yükle
        metadata = joblib.load(metadata_yolu)
        material_ids = metadata['material_ids']
        
        # Embeddings dosyasını yükle
        embeddings = np.load(embeddings_yolu, allow_pickle=True)
        
        # Eğer dosya bir nesne dizisi ise, onu uygun şekilde işleyin.
        if embeddings.ndim == 1:
            embeddings_dizi = np.vstack(embeddings)
        else:
            embeddings_dizi = embeddings

        print(f"Part 23 yüklendi. Material IDs: {len(material_ids)}, Embeddings: {embeddings_dizi.shape}")
        
        # DataFrame'i oluştur
        df_part_23 = pd.DataFrame(embeddings_dizi)
        df_part_23.insert(0, 'material_id', material_ids)

        print("\nDataFrame başarıyla oluşturuldu.")
        print(f"DataFrame'in şekli: {df_part_23.shape}")
        print("\nDataFrame'in ilk 5 satırı:")
        print(df_part_23.head())

        # İsteğe bağlı olarak DataFrame'i CSV dosyasına kaydedebilirsiniz
        cikti_yolu = os.path.join(dosya_klasoru, 'embeddings_part_23.csv')
        df_part_23.to_csv(cikti_yolu, index=False)
        # print(f"\nDataFrame '{cikti_yolu}' olarak kaydedildi.")

    except Exception as e:
        print(f"Hata oluştu: Part 23 yüklenemedi. Hata: {e}")
else:
    print("Part 23 dosyaları bulunamadı. Lütfen dosya yollarını kontrol edin.")

Part 23 yüklendi. Material IDs: 120064, Embeddings: (120064, 96)

DataFrame başarıyla oluşturuldu.
DataFrame'in şekli: (120064, 97)

DataFrame'in ilk 5 satırı:
  material_id    0    1    2         3    4    5    6    7    8  ...  \
0       mp-10  0.0  0.0  0.0  0.591774  0.0  0.0  0.0 -0.0  0.0  ...   
1      mp-100  0.0  0.0  0.0 -0.761594  0.0  0.0  0.0  0.0  0.0  ...   
2     mp-1000  0.0  0.0  0.0  0.542677  0.0  0.0  0.0 -0.0  0.0  ...   
3    mp-10000  0.0  0.0  0.0  0.519970  0.0  0.0  0.0  0.0  0.0  ...   
4    mp-10003  0.0  0.0  0.0  0.385869  0.0  0.0  0.0  0.0  0.0  ...   

          86         87         88         89         90         91  \
0   2.484530  -9.632307 -32.394093  41.661289 -28.114471  46.717102   
1  14.894664   0.378117 -57.083275  60.759682 -29.902206  91.257156   
2  26.224220 -12.049005 -56.753151  76.408203 -33.990269  70.319084   
3  10.380583   6.374668 -46.124512  60.831505 -30.181965  86.349068   
4  14.546944   1.860483 -47.701294  74.198524 -48.88